In [ ]:
%matplotlib inline
import os

import torch
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, models

from torchsummary import summary

device = torch.device('cuda:1')

In [ ]:
# fileid: 12XWUcct4LA_bZXaccScYUBC97S9R90L1
# filename: crop_part1.tar.gz
!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=12XWUcct4LA_bZXaccScYUBC97S9R90L1' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=12XWUcct4LA_bZXaccScYUBC97S9R90L1" -O crop_part1.tar.gz && rm -rf ~/cookies.txt
!tar -zxvf crop_part1.tar.gz

In [ ]:
tmp = os.listdir('./crop_part1')
print(len(tmp))
print(tmp[0])

In [ ]:
fname = '1_1_0_20161219204750596.jpg.chip.jpg'
splits = fname.split('_')
print(splits)

In [ ]:
img = Image.open(
    './crop_part1/1_1_0_20161219204750596.jpg.chip.jpg').convert('RGB')
img

In [ ]:
transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

class FaceDataset(Dataset):
    def __init__(self, root_dir, transform):
        # implement this.
        # 필요한 정보들 생성.
        # Hint: os.listdir(directory) -> List[filename]
        
    def __len__(self):
        # implement this.
        # 데이터 셋의 크기가 얼마인지? (또는, image가 몇장 있는지)
        return 
        
    def __getitem__(self, idx):
        # implement this
        # 인덱스 (idx)를 받아서, 해당 idx 의 data 와 label 을 반환.
        # Hint: Image.open(filename).convert('RGB')
        # Hint: '1' -> 1. 로 바꾸기 위해선 float() 을 사용.
        # Hint: filename = {age}_{gender}_{race}_{time}.jpg
        return {
            "image": img, # [3, 224, 224]
            "age": age,
            "gender": gender,
            "race": race,
            "filename": filename
        }

In [ ]:
##implement dataset and dataloader
dataset = FaceDataset("./crop_part1", transform)

In [ ]:
def my_collate_fn(list_data):
#     [dataset.__getitem__(0), ..., dataset.__getitem__(7)]

    print("My collate function is working!!!!")
    data_dicts = {
        "image": [], # torch tensor [8, 3, 512, 512]
        "age": [], # torch tensor [8]
        "gender": [], # torch tensor [8]
        "race": [], # torch tensor [8]
        "filename": [] # torch tensor [8]
    }
    
    batch_size = len(list_data)
    for i in range(batch_size):
        data = list_data[i] # {"image": torch.Tensor, "age": float, "filename": str}


    return data_dicts

In [ ]:
batch_size = 8
trainloader = DataLoader(
    train_set, batch_size = batch_size, shuffle = True, collate_fn=my_collate_fn
)
testloader = DataLoader(
    test_set, batch_size = batch_size, shuffle = False, collate_fn=my_collate_fn
)

In [ ]:
# display some images

import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img / 2 + 0.5  # Unnormalize
    npimg = img.numpy()
    plt.figure(figsize=(20,12))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
    
# get some random training images
dataiter = iter(trainloader)
data_dicts = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(data_dicts["image"]))

# print labels
print('Age : ', ' '.join('%5s' % str(data_dicts["age"][j].item()) for j in range(8)))
print('Gender : ', ' '.join('%5s' % str(data_dicts["gender"][j].item()) for j in range(8)))
print('Race : ',' '.join('%5s' % str(data_dicts["race"][j].item()) for j in range(8)))

In [ ]:
# ResNet18 pretrained network 받아서 씀
class Net(nn.Module):
    def __init__(self, use_pretrained=True):
        super(Net, self).__init__()
        ##implement this self.net, self.net.fc change into in_feature size
        # [3, 224, 224]
        
        
    def forward(self, x):
        ##implement this


In [ ]:
net = models.vgg16()
print(net.classifier[6])

In [ ]:
#implement criterion
criterion = nn.MSELoss()

model = Net(use_pretrained=True)
summary(model, batch_size=-1, input_size=(3, 224, 224), device='cpu')
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
def train(epoch):
    model.train()
    total_loss = 0.
    for batch_idx, data_dict in enumerate(trainloader):
        target = data_dict["age"].type(torch.float).view(-1, 1)
#         target = target.type(torch.float).view(-1,1)
        data = data_dict["image"].to(device)
        target = target.to(device)
#         data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), total_loss/(batch_idx+1)))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data_dict in testloader:
        target = data_dict["age"].type(torch.float).view(-1,1)
        data, target = data_dict["image"].to(device), target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item() # sum up batch loss
        

    test_loss /= len(testloader.dataset)
    print('\nTest set: Average loss: {:.4f}'.format(
        test_loss, correct, len(testloader.dataset)))

In [ ]:
for epoch in range(5):
    train(epoch)
    test()

In [ ]:
dataiter = iter(testloader)
data_dict = dataiter.next()
images = data_dict['image']

imshow(torchvision.utils.make_grid(images))
print('GrondTruth: ', ' '.join('%5s' % data_dict['age'][j].item() for j in range(8)))

In [ ]:
images = images.to(device)
outputs = model(images)

outputs = outputs.squeeze()

print('Predicted: ', ' '.join('%.1f' % outputs[j].item() for j in range(8))) 